集思录可转债页面的抓取

要点：
*.通过Chrome找到正确的链接

*.这里是Json格式，不是巨潮信息的html，所以需要解析json

*.正则,bs4,jsonpath,xpath.的选择  json数据优先选择使用jsonpath. html页面个人比较喜欢使用xpath,若使用xpath较难提取的数据可以使用bs4进行辅助, 若二者都提取不到,这时再去考虑使用正则

*. 用dict_json = json.loads(html) 转为字典

*. 存在Javascript动态生成的情况，还是用Selenium好


In [1]:
#coding:utf8

import urllib
import urllib.request
from bs4 import BeautifulSoup
import pandas as pd


import requests
import unittest
from bs4 import BeautifulSoup

import urllib3
from selenium import webdriver
import time

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys

from selenium.webdriver.common.action_chains import ActionChains

import socket

# import urllib2

#json解析库,对应到lxml
import json

#json的解析语法，对应到xpath
# import jsonpath

In [2]:
#抓取表格的Headers
# url='https://www.jisilu.cn/data/cbnew/#cb'
# req = urllib.request.Request(url)
# req.add_header('User-Agent','Mozilla/5.0 (Windows NT 6.2; rv:16.0) Gecko/20100101 Firefox/16.0')

# page = urllib.request.urlopen(req)

# html = page.read().decode('utf-8')

# soup =BeautifulSoup(html, 'lxml')
# soup


hostname=socket.gethostname()
if hostname=='MyWorkstation':
    profileDir = "C:/Users/Aaron/AppData/Roaming/Mozilla/Firefox/Profiles/yn80ouvt.default" #用于MYWorkStation
elif hostname=='MyMacWin10':
    profileDir = "C:/Users/Aaron/AppData/Roaming/Mozilla/Firefox/Profiles/ewg4x7y5.default-release" #用于MyMacwin
elif hostname=="DESKTOP-DEFM935":
    a=1
    

profile =webdriver.FirefoxProfile(profileDir)
driver = webdriver.Firefox(profile)
driver.implicitly_wait(5)

driver.get('https://www.jisilu.cn/data/cbnew/#cb')

time.sleep(5)

ht = driver.page_source
ht=ht.encode('gb18030').decode('gb18030')

In [4]:
soup =BeautifulSoup(ht, 'lxml')
soup

<html><head>
<meta content="text/html;charset=utf-8" http-equiv="Content-Type"/>
<meta content="Chrome=1,IE=edge" http-equiv="X-UA-Compatible"/>
<title>可转债 - 可转债 - 集思录</title>
<meta content="集思录,低风险理财,债券,公司债,国债,可转债,A类,新股," name="keywords"/>
<meta content="集思录是一个低风险投资理财问答社区" name="description"/>
<base href="https://www.jisilu.cn/"/>
<link href="https://www.jisilu.cn/static/js/tablesorter/themes/blue/style.css?v=20190909" rel="stylesheet" type="text/css"/>
<link href="https://www.jisilu.cn/static/css/default/bootstrap3/bootstrap.min.css?v=20190909" rel="stylesheet" type="text/css"/>
<link href="https://www.jisilu.cn/static/css/default/jisilu_framework.css?v=20190909" rel="stylesheet" type="text/css"/>
<link href="https://www.jisilu.cn/static/css/default/bootstrap3/bootstrap-datepicker3.min.css?v=20190909" rel="stylesheet" type="text/css"/>
<script src="//hm.baidu.com/hm.js?164fe01b1433a19b507595a43bf58262"></script><script type="text/javascript">
var _D40C9052107839F8DF0D875EDB2A628C='';

In [20]:
##########找到表格头
thead=soup.find('thead').find_all('tr')[1].find_all('th')#.find("div").find_all('title')
# print(thead)
headcolumnlist=[]
for each in thead:
#     print(each)
    headcolumnlist.append(each.text)
#     print('\n')
print(headcolumnlist)

columnnumbers=len(headcolumnlist)
columnnumbers

['代\xa0码', '转债名称', '现\xa0价', '涨跌幅', '正股名称', '正股价', '正股涨跌', 'PB', '转股价', '转股价值', '溢价率', '纯债价值', '评级', '期权价值', '回售触发价', '强赎触发价', '转债占比', '机构持仓', '到期时间', '剩余年限', '到期税前收益', '到期税后收益', '回售收益', '成交额(万元)', '操作']


25

In [27]:
#创建一个空DataFrame，然后插入row的方式
df=pd.DataFrame(columns=headcolumnlist)
df

,代 码,转债名称,现 价,涨跌幅,正股名称,正股价,正股涨跌,PB,转股价,转股价值,...,强赎触发价,转债占比,机构持仓,到期时间,剩余年限,到期税前收益,到期税后收益,回售收益,成交额(万元),操作


In [28]:
# 每行的数据作为新增加入DataFrame
dict1={}

trs=soup.find('table', id="flex_cb").find('tbody').find_all('tr')
for each in trs: #每行
#     print(each)
    for i in range(len(headcolumnlist)):
        dict1[headcolumnlist[i]]=each.find_all('td')[i].text
        
    df=df.append(dict1,ignore_index=True)

df

,代 码,转债名称,现 价,涨跌幅,正股名称,正股价,正股涨跌,PB,转股价,转股价值,...,强赎触发价,转债占比,机构持仓,到期时间,剩余年限,到期税前收益,到期税后收益,回售收益,成交额(万元),操作
0,123031,晶瑞转债,154.089,4.26%,晶瑞股份,32.70,1.21%,8.85,18.380,177.91,...,23.89,3.7%,会员,25-08-28,5.715,-4.90%,-5.36%,购买,14610.83,
1,113539,圣达转债,125.750,1.22%,圣达生物,38.80,-0.77%,4.93,28.780,134.82,...,37.41,6.9%,会员,25-07-02,5.559,-0.62%,-1.27%,购买,2649.37,
2,113538,安图转债,146.540,-0.68%,安图生物 R,98.52,-0.98%,17.83,64.110,153.67,...,83.34,1.7%,会员,25-06-27,5.545,-4.33%,-4.78%,购买,1307.00,
3,128020,水晶转债,138.000,1.85%,水晶光电 R,17.07,2.40%,4.59,12.230*,139.57,...,15.90,4.9%,会员,23-11-16,3.932,-5.43%,-5.91%,购买,15022.45,
4,128069,华森转债,108.080,-0.02%,华森制药,19.76,-2.52%,7.66,18.080,109.29,...,23.50,3.8%,会员,25-06-24,5.537,2.02%,1.37%,购买,1296.50,
5,128068,和而转债,137.990,3.74%,和而泰 R,12.68,3.76%,5.79,9.090,139.49,...,11.82,4.8%,会员,25-06-04,5.482,-3.60%,-4.01%,购买,10232.92,
6,128041,盛路转债,150.013,-0.79%,盛路通信 R,10.31,-0.67%,2.51,6.850,150.51,...,8.90,0.8%,会员,24-07-16,4.597,-5.72%,-6.25%,购买,110.30,
7,123022,长信转债,153.158,3.28%,长信科技 R,9.60,2.89%,4.01,6.250,153.60,...,8.12,3.1%,会员,25-03-18,5.268,-5.34%,-5.82%,购买,27548.01,
8,123003,蓝思转债,134.778,0.73%,蓝思科技 R,14.11,1.51%,3.25,10.440**,135.15,...,13.57,3.4%,会员,23-12-08,3.992,-5.08%,-5.50%,购买,6440.46,
9,128052,凯龙转债,178.490,-0.61%,凯龙股份 R,12.10,-0.49%,2.61,6.770,178.73,...,8.80,1.6%,会员,24-12-21,5.030,-8.45%,-8.92%,购买,2992.79,


In [34]:
df.to_csv('c:/jisilu_cbond.csv',encoding='utf-8-sig')

In [30]:
# url='https://www.jisilu.cn/data/cbnew/#cb'
# url='https://www.jisilu.cn/data/cbnew/cb_index_quote/'
# url='https://www.jisilu.cn/data/cbnew/cb_list/?___jsl=LST___t=1576123233361'
# req = urllib.request.Request(url)
# req.add_header('User-Agent','Mozilla/5.0 (Windows NT 6.2; rv:16.0) Gecko/20100101 Firefox/16.0')

# page = urllib.request.urlopen(req)

# html = page.read().decode('utf-8')
# html

# unicodestr=json.loads(html)

In [31]:
# soup =BeautifulSoup(html, 'lxml')
# soup

# unicodestr
# dict_json = json.loads(html)  
# dict_json

In [32]:
# total=dict_json['total']

# df=pd.DataFrame(dict_json['rows'][0])
# df

NameError: name 'dict_json' is not defined

In [33]:
# for i in range(total):
#     firstrow=dict_json['rows'][i]
#     print(firstrow)
#     print("\n")